# Upskills clustering

A number of experiments on trying to make sense of the ~200 job offers compiled as part of the upskills project. 
The core idea is trying to organise the kib offers. Since we do not have any annotation, we have opted for carrying out a clustering. Both representations and clustering come from [scikit](https://scikit-learn.org/) (twiking a bit the tokenizer)

**Representations**


* [TF-IDF](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
* [SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) (LSA)

I had considered doc2vec at first, but the computation was taking way too long

**Clustering alternatives**
* [Birch](https://scikit-learn.org/stable/modules/clustering.html#birch). Requires the number of clusters. 
* [Afifinity propagation](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation). Estimates a reasonable number of clusters.
* [Meanshift](https://scikit-learn.org/stable/modules/clustering.html#mean-shift).  Estimates a reasonable number of clusters.

**Requirements (non-python-standard)**
* spacy 3.0. Tokenization, lemmatization
* sklearn. Feature computation, clustering
* pandas. Dataframes 

In [1]:
# Downloading the model for spacy 
# RUN ONLY IF YOU DON'T HAVE THE MODELS READY
! python3 -m spacy download en_core_web_sm
# ! python -V
# ! which python
# ! pip3 install --upgrade --upgrade-strategy eager sklearn
# ! pip3 install --upgrade spacy

     |████████████████████████████████| 13.7 MB 1.3 MB/s eta 0:00:011
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# checking the version of spacy (don't run)
import spacy as kk
  
# Check the version 
print(kk.__version__) 

3.0.3


In [3]:
# import gensim.models as g
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

import spacy 

# from spacy.tokenizer import Tokenizer
# from spacy.lang.en import English

from sklearn.decomposition import PCA, TruncatedSVD
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import AffinityPropagation
from sklearn.cluster import Birch
from sklearn.cluster import MeanShift, estimate_bandwidth

from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.preprocessing import MinMaxScaler


#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

HERE ARE THE CONSTANTS THAT YOU MIGHT WANT TO MODIFY

In [4]:
# the path to the corpus
path = "/Users/albarron/tmp/adriano/txt_raw"

# the number of PCA components (e.g., 2, 20, 100)
PCA_COMPONENTS = 100
SVD_COMPONENTS = 100
# From when considering to use doc2vec. Not really necessary
# model = "/Users/albarron/corpora/embeddings/doc2vec/enwiki_dbow/doc2vec.bin"
#load model
# m = g.Doc2Vec.load(model)

In [5]:
def find_files(path):
    """Loads all the txt files in the path folder and 
    returns them, with their full path"""
    my_files = []
    for root, dirs, files in os.walk(path):
        my_files.extend(os.path.join(path, file) for file in files if file.endswith(".txt"))
    return my_files

In [6]:
def extract_xml(file):
    """Extract the contents from an xml file (which in this 
    corpus actually have txt extension) and returns it as 
    a dictionary. 
    The assumed tags are: 'id', 'jobtitle', 'about', 'jobdesc', 
    'keyinfo', 'benefits'.
    If jobdesc contains internal tags, all their text is simply 
    merged.
    """
    with open(file) as f:
        elements = {}
        tree = ET.fromstring(f.read())
        elements['id'] = tree.attrib['id']

        for child in tree: #ET.fromstring(f.read()):
            # elements[child.tag] = child.text
            elements[child.tag] = ' '.join(child.itertext()).lower()
    return elements

## Load all the files into a pandas dataframe

In [7]:
files = find_files(path)
df = pd.DataFrame(
    columns=['id', 'jobtitle', 'about', 'jobdesc', 'keyinfo', 'benefits'])

for file in files:
#     print(file)
    d = extract_xml(file)
    df = df.append(d, ignore_index=True)

# replace empty jobtitles (NAN in a dataframe) with ''
df.jobtitle = df.jobtitle.fillna('')
# Add a new column combining offer title and description
df['jobtitle_desc'] = " " + df.jobtitle + '\n' + df.jobdesc
df.set_index('id')
# print(df.jobdesc)
print(df.jobtitle_desc)

0       localization editor,\njapanese\nlanguages are...
1       german; english; applied linguistics; computa...
2       english; computational linguistics; general l...
3        english; french; computational linguistics; ...
4       english; computational linguistics: senior an...
                             ...                        
193     french canadian linguist - siri tts\napple is...
194      english; german; spanish; applied linguistic...
195     french; german; syntax: analytical linguist, ...
196     swedish; computational linguistics; lexicogra...
197     senior social analyst, data\noverview\nthe id...
Name: jobtitle_desc, Length: 198, dtype: object


## Representations production

### Alternative 1: tf-idf

In [8]:
# nlp = English()
# tokenizer = Tokenizer(nlp.vocab)

# nlp.add_pipe("lemmatizer", config={"mode": "lookup"})
nlp = spacy.load("en_core_web_sm")
# config = {"mode": "rule"}
# nlp.add_pipe("lemmatizer", config=config)
# This usually happens under the hood
# processed = nlp(doc)


doc = nlp("this is a very small tests trying to check if I am or you are actually getting lemmas and not tokens")
# lemmas = set([w.lemma for w in doc])
# tokens = set([w for w in doc])
# print (len(lemmas), len(tokens))
# print(doc)
print([token.lemma_ for token in doc])


['this', 'be', 'a', 'very', 'small', 'test', 'try', 'to', 'check', 'if', 'I', 'be', 'or', 'you', 'be', 'actually', 'get', 'lemmas', 'and', 'not', 'token']


In [9]:
# tfidf_docs = pd.DataFrame(tfidf_docs)
# # # centers the vectorized documents (BOW vectors) by subtracting the mean
# # tfidf_docs = tfidf_docs - tfidf_docs.mean()

def lemmatized_words(doc):
    """A tokenizer based on spacy to add lemmas to the vector, 
    rather than tokens. It also ignores sequences of spaces 
    and 1-character tokens"""
    doc = nlp(doc)
    return (w.lemma_ for w in doc if w.lemma_.strip() !="" and len(w.lemma_) > 1)

# Without lemmatization; the vocabulary is huge: 75.4k.
# lemm_vectorizer = TfidfVectorizer(tokenizer=tokenizer) 

# With lemmatization; the vocabulary is reasonable < 3k
lemm_vectorizer = TfidfVectorizer(
    tokenizer=lemmatized_words, # the tokenizer from the function 
    stop_words='english', 
    min_df=2)

tfidf_docs = lemm_vectorizer.fit_transform(raw_documents=df.jobtitle_desc).toarray()
print("Size of the vocabulary:", len(lemm_vectorizer.vocabulary_))

tfidf_docs = pd.DataFrame(tfidf_docs)
print("Shape of the matrix:", tfidf_docs.shape)

# Uncomment if you want to see the features (vocabulary)
# print("Feature names:\n", lemm_vectorizer.get_feature_names())

# print(tfidf_docs)

/usr/local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ca', 'far', 'make', 'nt'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Size of the vocabulary: 2317
Shape of the matrix: (198, 2317)


## Alternative 2: PCA

In [10]:
# Scaling the tfidf vectors in [0,1] (recommended for PCA)
scaler = MinMaxScaler()
tfidf_docs_rescaled = scaler.fit_transform(tfidf_docs)

# Computing the PCS with PCA_COMPONENTS
pca = PCA(svd_solver = 'full', n_components=PCA_COMPONENTS)
pca = pca.fit(tfidf_docs_rescaled)
pca_topic_vectors = pca.transform(tfidf_docs_rescaled)

columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=df.id)
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
id,,,,,,,,,,,,,,,,,,,,,
Facebook001,-0.739,-0.140,-0.029,-0.298,0.086,-0.097,-0.921,-0.159,0.174,0.017,...,0.663,-0.073,0.767,-0.612,0.396,0.298,0.130,-0.419,-0.045,-0.238
Linguist014,0.504,-0.772,-0.043,-0.135,-0.570,0.361,-0.030,0.464,0.551,-0.382,...,-0.501,-0.159,-0.082,0.293,0.266,0.241,0.037,0.089,0.129,0.098
Linguist028,0.826,-0.059,0.449,-0.100,-0.070,0.978,-1.857,-1.199,-0.666,-1.143,...,0.032,-0.070,-0.191,-0.004,0.092,-0.176,-0.368,0.123,-0.286,-0.156
Linguist029,0.971,-0.068,0.295,-0.180,-0.228,1.131,-1.231,-0.983,-1.063,-1.616,...,0.302,0.911,0.032,0.452,-0.065,0.039,0.653,0.371,0.186,-0.149
Linguist015,-0.306,-0.476,-1.245,3.051,1.819,0.225,-0.562,-0.439,-0.274,0.208,...,-0.542,-0.015,-0.234,0.227,0.262,-0.034,-0.400,0.117,0.164,0.083
Linguist001,1.071,-0.269,0.009,-0.011,-0.400,0.806,-0.168,1.113,-0.048,0.447,...,-0.003,0.450,-0.062,0.401,-0.301,-0.158,0.227,-0.243,0.041,-0.125


## Alternative 3: TruncatedSVD (LSA)

In [11]:
svd = TruncatedSVD(n_components=SVD_COMPONENTS)   #, n_iter=100)
scaler = MinMaxScaler()
tfidf_docs_rescaled = scaler.fit_transform(tfidf_docs)

# Decomposes TF-IDF vectors and transforms them into topic vectors
svd_topic_vectors = svd.fit(tfidf_docs_rescaled)
svd_topic_vectors = svd.transform(tfidf_docs_rescaled)

# print(svd_topic_vectors.shape)
columns = ['topic{}'.format(i) for i in range(svd.n_components)]
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=df.id)

# Display the top-6 vectors
svd_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
id,,,,,,,,,,,,,,,,,,,,,
Facebook001,1.612,-0.715,-0.102,0.091,-0.315,-0.006,-0.043,-0.893,-0.018,0.212,...,0.059,0.763,-0.211,-0.308,0.196,-0.311,0.485,0.015,-0.280,0.172
Linguist014,1.695,0.583,-0.379,0.441,0.012,-0.785,0.263,-0.061,0.691,0.208,...,-0.100,-0.147,0.417,0.123,-0.003,-0.144,-0.188,0.109,-0.308,0.447
Linguist028,2.210,0.785,0.336,0.319,-0.110,-0.283,0.985,-1.768,-1.480,0.988,...,0.265,0.191,0.240,0.040,-0.251,0.090,0.163,-0.119,-0.310,-0.048
Linguist029,2.313,0.920,0.202,0.237,-0.141,-0.537,1.049,-1.182,-1.602,0.708,...,0.353,-0.250,-0.640,0.158,0.598,0.410,0.514,0.574,-0.038,-0.608
Linguist015,2.483,-0.438,-1.697,0.804,2.130,2.393,0.430,-0.509,-0.457,-0.135,...,-0.132,-0.233,0.214,0.009,-0.162,-0.004,-0.121,-0.007,-0.377,0.283
Linguist001,2.057,1.055,-0.118,0.187,0.079,-0.626,0.638,-0.269,0.771,-0.868,...,-0.202,-0.724,-0.269,-0.169,-0.073,-0.448,0.157,0.096,0.590,0.157


# Clustering

Notice that running every alternative would add the column with the clusters to the df (and would be included in the saved tsv)

## Alternative 1: Birch (which requires the number of clusters)

In [12]:
def birch(data, k):
    "Produces a clustering with k clusters for the given data"
    brc = Birch(branching_factor=50, n_clusters=k, threshold=0.1, compute_labels=True)
    brc.fit(data)

    clusters = brc.predict(data)

    labels = brc.labels_
    return clusters

    
    # print ("Clusters: ")
#     print (clusters)
# print(df.head(10))


### Birch with tfidf

In [13]:
for k in range(1, 21):
    clusters = birch(tfidf_docs, k)
    df[".".join(["birch", "tfidf", str(k)])] = clusters 

### Birch with PCA

In [14]:
for k in range(1, 21):
    clusters = birch(pca_topic_vectors, k)
    df[".".join(["birch", "pca", str(k)])] = clusters 

### Birch with SVD

In [15]:
for k in range(1, 21):
    clusters = birch(svd_topic_vectors, k)
    df[".".join(["birch", "svd", str(k)])] = clusters 

## Alternative 2: [AffinityPropagation](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation)

In [16]:
def aff_propagation(X):
    """Compute clusters based on Affinity Propagation"""
    af = AffinityPropagation(random_state=None)
    af.fit(X)
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_

    n_clusters_ = len(cluster_centers_indices)

    print('Estimated number of clusters: %d' % n_clusters_)
    # print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
    # print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
    # print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
    # print("Adjusted Rand Index: %0.3f"
    #       % metrics.adjusted_rand_score(labels_true, labels))
    # print("Adjusted Mutual Information: %0.3f"
    #       % metrics.adjusted_mutual_info_score(labels_true, labels))
    # print("Silhouette Coefficient: %0.3f"
    #       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
    # print("Silhouette Coefficient: %0.3f"
    #       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
    print(labels)
    return labels
#     print(labels)

### AffinityPropagation with tfidf

In [17]:
# centers the vectorized documents (BOW vectors) by subtracting the mean
X = tfidf_docs
X = X - X.mean()
clusters = aff_propagation(X)
df["affprop.tfidf"] = clusters 

Estimated number of clusters: 40
[ 0 13  1  5 19 26  0  3  7  6 11 27 39 13 26 26  9 31  3  2  9 31 26  3
  7 19  3 26  4 17  3 27  5  7 15 14 31  0 33  6 35  4  7  5 31  7 10  7
 10 10 25 37 30  0 39  6 35  8  9 10 11 26 10 12 13  9 35 14 14  4 15 34
 15 16 17  3 39  0 11  5 10 16  6 16 31  4 23 26 27 12 31 18 31 27 38 16
 21 19 25 24 20 16 32 32 23 36  4 21 30 21 18 31  1 22 27 14 23 37 10 31
 24 25 36 31 18 30 38 26 31  7 34 38  1 27 22  2 31 37  7 34 28 31  7 25
 17 29 13  9 31 28 39 30 30 20 31 34 33 32 23 10 33  8  3 29 26  3  7  2
  7 10 21 19 10 17 19 25  4 34 35 27 26 34 26 10 28 36 19 26  5  3 27 38
 36 37 26 24 38 39]


### AffinityPropagation with PCA

In [18]:
clusters = aff_propagation(pca_topic_vectors)
df["affprop.pca"] = clusters 

Estimated number of clusters: 25
[ 2  0 13  4  1  0  2 24  2  2  0  2  2 20 20  0  2  2  3  2  2  2  2  3
  0  1  3  2  2  2 24  2  4  2  8  2 20  2  0  2  2  2  2  4 20  2  2 20
  6  6  2 23  2  2  5  2 22 19 20  6  0 20  7  2  0 20 22  2  0  2  2 20
  8  2  0 20  2  2 20  4  2  2  2 20  2  2 20 20  2  0  2 20 12  2 24  0
  2 21 15 20  9  2 10  2 11 20  2  2 20  2 20 12 13 17  2  2 11 23 14  9
 20 15  2 16  2  2 24 20  2  2  0 24  2  2 17  0  2 23  2  2 18  9  2  0
 20 20  0  2  2 18  2  2  2  9  9  0 20 10  2  2  2 19 20 20  0 20  2  2
  2 20  2 21  2  0 21  2  2 20 22  2  0  0 20  7  0 20  2  0  2 20  2 20
  2 23 23 20 24  2]


### AffinityPropagation with SVD

In [19]:
clusters = aff_propagation(svd_topic_vectors)
df["affprop.svd"] = clusters 

Estimated number of clusters: 25
[22 19  0  4  1  2  2 24  2  2  3  2  2 19 22 22  2  2  3  2 22 22  2  3
 19  1  3  2  2 19 24  2  4  2  5  2 19 22  2  2 22  2  2  4 22  2  2 19
  8  8  2 23 22  2  6 22 21  7 19  8 19 19  9 22 10 19 21  2  2 22  2 19
  5  2 22 19 22  2 22  4  2 22  2 19  2  2 22 19  2 22  2 22 14  2 24 22
  2 20 15 19 11  2 12  2 13 19 22  2 19  2 22 14  0 17  2 22 13 23 22 11
 19 15  2 16  2  2 24 19 22  2 19 24  2  2 17 22 22 23  2  2 18 11 22 22
 22 19 10 22  2 18 22  2  2 11 11 22 19 12  2 22  2  7 19 19 22  3  2  2
  2  2 22 20  2 22 20  2  2 19 21 22 22 22 19  9 22 19  2 19  2 22 22 19
 19 23 23 22 24  2]


# Alternative 3: [Meanshift](https://scikit-learn.org/stable/modules/clustering.html#mean-shift)

In [20]:
def mean_shift(X):
    """Compute clusters based on Meanshift"""
    # The following bandwidth can be automatically detected using
    bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)

    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(X)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)

    print("number of estimated clusters : %d" % n_clusters_)
    return labels

### MeanShift with tfidf

In [21]:
X = tfidf_docs
X = X - X.mean()
clusters = mean_shift(X)
df["meanshift.tfidf"] = clusters

number of estimated clusters : 1


### MeanShift with PCA

In [22]:
clusters = mean_shift(pca_topic_vectors)
df["meanshift.pca"] = clusters 

number of estimated clusters : 1


### MeanShift with SVD

In [23]:
clusters = aff_propagation(svd_topic_vectors)
df["meanshift.svd"] = clusters 

Estimated number of clusters: 25
[21 18  0  4  1  2  2 24  2  2  3  2  2 18 21 21  2  2  3  2 21 21  2  3
 18  1  3  2  2 18 24  2  4  2  8  2 18 21  2  2 21  2  2  4 21  2  2 18
  7  7  2 23 21  2  5 21 20  6 18  7 18 18 22 21 16 18 20  2  2 21  2 18
  8  2 21 18 21  2 21  4  2 21  2 18  2  2 21 18  2 21  2 21  9  2 24 21
  2 19 10 18 11  2 17  2 12 18 21  2 18  2 21  9  0 13  2 21 12 23 21 11
 18 10  2 14  2  2 24 18 21  2 18 24  2  2 13 21 21 23  2  2 15 11 21 21
 21 18 16 21  2 15 21  2  2 11 11 21 18 17  2 21  2  6 18 18 21  3  2  2
  2  2 21 19  2 21 19  2  2 18 20 21 21 21 18 22 21 18  2 18  2 21 21 18
 18 23 23 21 24  2]


In [24]:
print(df.head(3))
# df.to_csv("/".join([path, "upskills_clusters.tsv"]), sep="\t")

            id                                           jobtitle  \
0  Facebook001                     localization editor,\njapanese   
1  Linguist014  german; english; applied linguistics; computat...   
2  Linguist028  english; computational linguistics; general li...   

                                               about  \
0  about the facebook company\nfacebook's mission...   
1                                                NaN   
2  we are an equal opportunity employer and value...   

                                             jobdesc  \
0  languages are key to our mission of bringing t...   
1  description:\n\nthis is an exciting opportunit...   
2  description:\n\nappen is the world's leading i...   

                                             keyinfo benefits  \
0                                                         NaN   
1  university or organization: nuance communicati...      NaN   
2  university or organization: appen\ndepartment:...      NaN   

            